In [4]:
import json

# Open the dataset JSON
with open("dataSets/ingredient_and_instructions.json", "r") as dataSet:
    data = json.load(dataSet)

recipie = "" # The individual recipie
recipie_list = [] # Where we will store the recipie for training

for meal in data:
    # Get the recipie name and its instructions
    meal_info = data[meal]
    recipie += meal.replace("-", " ").title() + "\n\n"
    recipe_instructions = meal_info["instructions"]

    # Get the ingredients
    ingredients = meal_info["ingredient_sections"]

    for i in ingredients:
        # Retrieve ingredients and their details
        item = ""           # Dough
        qty = ""            # ½
        unit = ""           # cup
        ingredient = ""     # unsalted butter
        primary_unit = { "quantity": None, "display": None }

        # Some ingredients arent named, but are still listed as "1 cup" or similar
        # Additionally, if there is an ingredient name, it may already have a colon
        # in front of it, so we need to check for that before formatting
        if i["name"]:
            if i["name"][-1] != ":": # If there isnt a colon, add one
                item = i["name"] + ": \n"
            else: # Otherwise, just add the name
                item = i["name"] + " \n"
        else:
            item = ""

        recipie += item # Add the ingredient name to the recipie

        # Loop over each ingredient
        j = 0
        while j < len(i["ingredients"]):
            # i["ingredients"][0] may not exist, use try/except to prevent
            # any ListIndexErrors from accessing an index that doesn't exist
            try: primary_unit = i["ingredients"][j]["primary_unit"]
            except: primary_unit = { "quantity": None, "display": None } # Dummy object

            # Some recipies don't list an ingredient at all
            try: ingredient += i["ingredients"][j]["name"]
            except: ingredient += ""

            # If the try/except blocks above are successful, we can set the values
            if primary_unit["quantity"]: qty = primary_unit["quantity"] + " "
            if primary_unit["display"]: unit = primary_unit["display"] + " "

            # Construct the ingredient string
            recipie += qty + unit  + ingredient + "\n"
            ingredient = ""
            j += 1

        recipie += "\n"

    # Newline before adding instructions
    recipie += "\n"

    # Loop over the instructions and format them
    step = 1
    for text in recipe_instructions:
        instruction = text["display_text"]
        recipie += str(step) + ". " + instruction + "\n"
        step += 1

    # Add the recipie to the list
    recipie_list.append(recipie)

    # Free memory for the next recipie
    recipie = ""
    recipe_instructions = []
    ingredients = []
    item = ""
    qty = ""
    unit = ""
    ingredient = ""

# For testing purposes, we can output a random recipie
import random
print(recipie_list[random.randint(0, len(recipie_list) - 1)])

#print(recipie_list[0])


Spicy White Bean Chili

1 yellow onion
2 tablespoons olive oil
1 can great northern beans
4 cloves garlic
2 teaspoons cumin
¼ teaspoon ground cloves
1 ½ teaspoons dried oregano
½ teaspoon cayenne pepper
1 cup mild green chillis
3 cups veggie broth
1 teaspoon salt
½ teaspoon pepper
2 tablespoons fresh parsley


1. In a slow cooker, combine all ingredients except the parsley.
2. Set to low heat for 8-10 hours. Keep in mind that the longer you cook, the spicier the chili will become! (Adjust the amount of chilis accordingly)
3. Sprinkle parsley over top, stir, and serve.
4. Enjoy!



In [5]:
#CSV File 1 Record per row
#Record: 1 string with ingredients + recipe


# [[recipe1], [recipe2], [recipe3]]

import csv

with open("recipe.csv", 'w', encoding="utf-8") as csvfile:
    csvwriter = csv.writer(csvfile)
    for recipie in recipie_list:
        csvwriter.writerow([recipie])


In [6]:
from aitextgen.TokenDataset import TokenDataset
from aitextgen.tokenizers import train_tokenizer
from aitextgen.utils import GPT2ConfigCPU
from aitextgen import aitextgen

file_name = "recipe.csv"

train_tokenizer(file_name)
tokenizer_file = "aitextgen.tokenizer.json"
config = GPT2ConfigCPU()
ai = aitextgen(tokenizer_file=tokenizer_file, config=config)
data = TokenDataset(file_name, tokenizer_file=tokenizer_file, block_size=64)
ai.train(data, batch_size=120, num_steps=64000, generate_every=1000, save_every=5000)
ai.generate(prompt="steak")

C:\Users\flash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are tokenizing a CSV file, but you did not set line_by_line=True. Please change if unintended.
100%|██████████| 9922/9922 [00:00<00:00, 13055.39it/s]
pytorch_model.bin already exists in /trained_model and will be overwritten!
Windows does not support multi-GPU training. Setting to 1 GPU.
C:\Users\flash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:151: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_c

  0%|          | 0/200000 [00:00<?, ?it/s]

C:\Users\flash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pytorch_lightning\trainer\trainer.py:2281: LightningDeprecationWarning: `trainer.progress_bar_dict` is deprecated in v1.5 and will be removed in v1.7. Use `ProgressBarBase.get_metrics` instead.
  rank_zero_deprecation(


1,000 steps reached: generating sample texts.                                               
 for 15 minutes.                                                                                    
4. Add the cheddar-sesame, egg, fold the cream, and bled corn, and chives.
5. Pour the water into the cream, and mix mixture.
12. Line a large baking panko
2,000 steps reached: generating sample texts.                                               
. Continue mixing on low heat to low speed. Add ¼ cup (200 ml) of flour to the prepared pan and microwave for 3–4 minutes, until after forms. Remove the pan from the heat and set aside.
2.
3,000 steps reached: generating sample texts.                                               
 the oven and twosem into lines.                                                                    
9. Bake for 18 to 30 minutes, or until browned.
10. Serve each shrimp rest of pizza sauce wings and top with your favorite otional sauce,
4,000 steps reached: generating samp

In [9]:
ai.generate(prompt="salad")

salads.
12. Return the salmon to a serving platter and top with the remaining 2 tablespoons of olive oil, the turkey, a sage, and lemon slices.
13. Enjoy!
Chicken Zucchini
